In [16]:
import pulp as pl
import pandas as pd
import numpy as np
from typing import List, Dict

print("Setup Complete")

Setup Complete


In [22]:
# Read csv
file = "input_payroll_list/payroll_list_sample.csv"
df_salary = pd.read_csv(file)
df_salary

,name,salary,deduction,net_salary_int
0,person1,10977.28,-4000.00,6977
1,person2,12799.98,-1500.00,11299
2,person3,13437.80,0.00,13437
3,person4,23435.37,0.00,23435
4,person5,13318.40,-1500.00,11818
5,person6,12130.16,-1500.00,10630
6,person7,11058.81,0.00,11058
7,person8,9753.69,-1500.00,8253
8,person9,12453.00,0.00,12453
9,person10,11013.69,-11013.00,0


In [23]:
def distribute_large_denominations(
    amount: float, remaining_denominations: Dict[int, int], denominations: List[int]
) -> Dict[int, int]:
    """
    Distribute larger denominations more evenly.
    """
    breakdown = {}
    remaining = amount
    large_denominations = [1000, 500]  # Add or remove denominations as needed

    for denom in large_denominations:
        if denom not in remaining_denominations:
            continue

        # Calculate the ideal number of this denomination to use
        ideal_count = remaining // denom

        # Use only a portion of the ideal count to ensure even distribution
        count = min(
            ideal_count, remaining_denominations[denom], max(1, ideal_count // 2)
        )

        if count > 0:
            breakdown[denom] = count
            remaining -= denom * count

    return breakdown


def optimize_breakdown(
    amount: float, remaining_denominations: Dict[int, int], denominations: List[int]
) -> Dict[int, int]:
    """
    Optimize the breakdown for the remaining amount after large denomination distribution.
    """
    prob = pl.LpProblem("Denomination_Breakdown", pl.LpMinimize)

    count_vars = {
        denom: pl.LpVariable(
            f"P{denom}x", 0, remaining_denominations[denom], pl.LpInteger
        )
        for denom in denominations
    }

    # Objective: Minimize the number of bills/coins used
    prob += pl.lpSum(count_vars[denom] for denom in denominations)

    # Constraint: Must sum to exact amount
    prob += pl.lpSum(denom * count_vars[denom] for denom in denominations) == amount

    # Solve the problem (Coin-or Branch and Cut Solver)
    prob.solve(pl.PULP_CBC_CMD(msg=False))

    # Get the breakdown for this amount
    breakdown = {
        denom: int(round(count_vars[denom].value() or 0))
        for denom in denominations
        if count_vars[denom].value() is not None and count_vars[denom].value() > 0  # type: ignore
    }

    return breakdown


def create_denomination_breakdown(
    amounts: List[float], total_denominations: Dict[int, int], denominations: List[int]
) -> List[Dict[int, int]]:
    """
    Create optimized denomination breakdowns with smart substitution between denominations.
    """
    breakdowns = []
    remaining_denominations = total_denominations.copy()

    # Sort amounts in descending order to handle larger amounts first
    amount_indices = sorted(range(len(amounts)), key=lambda k: amounts[k], reverse=True)

    for i in amount_indices:
        amount = amounts[i]
        if amount == 0:  # Skip zero amounts
            breakdowns.append((i, {d: 0 for d in denominations}))
            continue

        breakdown = distribute_large_denominations(
            amount, remaining_denominations, denominations
        )
        remaining_amount = amount - sum(
            denom * count for denom, count in breakdown.items()
        )

        if remaining_amount > 0:
            small_breakdown = optimize_breakdown(
                remaining_amount, remaining_denominations, denominations
            )
            for denom, count in small_breakdown.items():
                breakdown[denom] = breakdown.get(denom, 0) + count

        # Update remaining denominations
        for denom, count in breakdown.items():
            remaining_denominations[denom] -= count

        breakdowns.append((i, breakdown))

    # Reorder breakdowns to match original amount order
    breakdowns.sort()
    return [b[1] for b in breakdowns]


def create_df(
    denom_breakdowns: List[Dict[int, int]],
    amounts: List[float],
    names: List[str],
    denominations: List[int],
) -> pd.DataFrame:
    """
    Create a DataFrame from the denomination breakdowns with detailed analysis.
    """

    data = []

    for i, breakdown in enumerate(denom_breakdowns):
        row = {"name": names[i], "net_salary_int": int(round(amounts[i]))}

        # Add denomination columns
        for d in denominations:
            row[f"P{d}x"] = breakdown.get(d, 0)

        # Calculate totals and variance
        total_amount = sum(d * row[f"P{d}x"] for d in denominations)
        row["total_amount"] = total_amount
        row["variance"] = total_amount - row["net_salary_int"]
        data.append(row)

    df = pd.DataFrame(data)

    # Reorder columns
    denom_cols = [f"P{d}x" for d in denominations]
    col_order = ["name", "net_salary_int"] + denom_cols + ["total_amount", "variance"]
    return df[col_order].reset_index(drop=True)


# Run the analysis

if __name__ == "__main__":

    # Get amounts and names
    amounts = df_salary["net_salary_int"].tolist()
    names = df_salary["name"].tolist()

    # Available denominations

    total_denominations = {
        1000: 284,
        500: 98,
        100: 130,
        50: 50,
        20: 105,
        10: 18,
        5: 50,
        1: 100,
    }

    denominations = sorted(total_denominations.keys(), reverse=True)

    # Create denomination breakdowns
    denomination_breakdowns = create_denomination_breakdown(
        amounts, total_denominations, denominations
    )

    # Create main results DataFrame
    df_denom = create_df(denomination_breakdowns, amounts, names, denominations)

In [19]:
# Show results of DataFrame
df_denom

,name,net_salary_int,P1000x,P500x,P100x,P50x,P20x,P10x,P5x,P1x,total_amount,variance
0,"Cabangangan, Leon",6977,2,0,44,11,1,0,1,2,6977,0
1,"Galve, Edwin",11299,11,0,2,1,2,0,1,4,11299,0
2,"Malilin, Crispulo",13437,9,8,4,0,1,1,1,2,13437,0
3,"Mondala, Ronie",23435,17,12,4,0,1,1,1,0,23435,0
4,"Pulvera, Raul",11818,8,7,3,0,0,1,1,3,11818,0
5,"Sable, Lope",10630,10,0,6,0,1,1,0,0,10630,0
6,"Alidanio, Eric",11058,11,0,0,1,0,0,1,3,11058,0
7,"Aranduque, Federico",8253,8,0,2,1,0,0,0,3,8253,0
8,"Benitez, Felipe",12453,9,6,4,1,0,0,0,3,12453,0
9,"Bertiz, Nerry",0,0,0,0,0,0,0,0,0,0,0


In [24]:
def create_summary_df(total_denominations: dict, df_denom: pd.Series) -> pd.DataFrame:

    denom = total_denominations.keys()
    given_notes = total_denominations.values()
    computed_notes = [df_denom[f"P{d}x"].sum() for d in total_denominations.keys()]
    variance = np.array(list(computed_notes)) - np.array(list(given_notes))

    # Create a dictionary with the data
    data = {
        "Denomination": denom,
        "Given Notes": given_notes,
        "Computed Notes": computed_notes,
        "Variance": variance,
    }

    # Create a DataFrame
    df = pd.DataFrame(data)
    return df


create_summary_df(total_denominations, df_denom)  # type: ignore

,Denomination,Given Notes,Computed Notes,Variance
0,1000,284,285,1
1,500,98,98,0
2,100,130,130,0
3,50,50,28,-22
4,20,105,19,-86
5,10,18,11,-7
6,5,50,15,-35
7,1,100,51,-49


In [25]:
# Save as csv
df_denom.to_csv(
    "output_payroll_list/output_payroll_list.csv",
    index=True,
)